In [11]:
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import confusion_matrix , accuracy_score
mlflow.set_tracking_uri('http://localhost:5000/')
ann = mlflow.set_experiment('ANN-bank')

In [18]:
%store -r df

In [19]:
%store -r X

In [20]:
y = df[['Exited']]

In [21]:
X.shape , y.shape

((10000, 13), (10000, 1))

In [22]:
def split_data(X ,y):    
    X_train ,  x_test,y_train ,y_test = train_test_split(X,y, test_size=0.2, random_state=0)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    x_test = sc.fit_transform(x_test)
    return X_train , x_test , y_train , y_test

In [23]:
X_train , x_test , y_train , y_test = split_data(X, y)

In [17]:
X_train.shape , x_test.shape , y_train.shape , y_test.shape

((8000, 13), (2000, 13), (8000, 1), (2000, 1))

In [48]:
def build_classifier(X_train,y_train,x_test,y_test):
    with mlflow.start_run(experiment_id=ann):

        classifier = Sequential()
        classifier.add(Dense(output_dim=6 , kernel_initializer='uniform',activation='relu' , input_dim=13))
        classifier.add(Dense(output_dim=6 , kernel_initializer='uniform',activation='relu'))
        classifier.add(Dense(output_dim=1 , kernel_initializer='uniform' , activation='sigmoid'))
        classifier.compile(optimizer='adam', loss='binary_crossentropy' , metrics=['accuracy'])

        classifier.fit(X_train,y_train, batch_size=10 , epochs=100, verbose=2)
        y_pred = classifier.predict(x_test)
        y_pred = y_pred > 0.5
        cm = confusion_matrix(y_test,y_pred)
        accuracy = accuracy_score(y_test , y_pred) * 100
        mlflow.log_metric("accuracy", accuracy)
        print('Confusion Matrix:',cm)

        mlflow.sklearn.log_model(classifier, "ANN-Bank")

In [ ]:
build_classifier(X_train , x_test , y_train , y_test)

In [24]:
def build_kfold_classifier():
        classifier_kfold= Sequential()
        classifier_kfold.add(Dense(output_dim=6 , kernel_initializer='uniform',activation='relu' , input_dim=13))
        classifier_kfold.add(Dense(output_dim=6 , kernel_initializer='uniform',activation='relu'))
        classifier_kfold.add(Dense(output_dim=1 , kernel_initializer='uniform' , activation='sigmoid'))
        classifier_kfold.compile(optimizer='adam', loss='binary_crossentropy' , metrics=['accuracy'])
        return classifier_kfold

In [31]:
with mlflow.start_run(experiment_id=ann):
        classifier_kfold_wrapper = KerasClassifier(build_fn=build_kfold_classifier , 
                                                   batch_size=10 , epochs=20, verbose=2)
        accu = cross_val_score(estimator=classifier_kfold_wrapper , 
                                X=X_train , y=y_train , cv=10 , verbose=1 , n_jobs=-1)
        
       
        
        
        mlflow.sklearn.log_model(classifier_kfold_wrapper, "ANN-Bank")

    

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.5min finished
